In [1]:
import os

In [2]:
%pwd

'/Users/mukulagarwal/Desktop/Projects/datascienceproject/research'

In [3]:
cd ..

/Users/mukulagarwal/Desktop/Projects/datascienceproject


/Users/mukulagarwal/Desktop/Projects/datascienceproject/prjt3/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
%pwd

'/Users/mukulagarwal/Desktop/Projects/datascienceproject'

In [12]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class DataValidationConfig:
    root_dir:Path
    unzip_data_dir:Path
    STATUS_FILE:str
    all_schema:dict

In [6]:
import pandas as pd

In [10]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.info()
list(data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [13]:
import os
from src.datascience.utils.common import read_yaml, create_directories 
from src.datascience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
class Configuration_Manager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
        
    # def get_data_ingestion_config(self) -> DataIngestionConfig:
    #     create_directories([self.config.data_ingestion.root_dir])
    #     config = self.config.data_ingestion
        
    #     data_ingestion = DataIngestionConfig(
    #         root_dir=config.root_dir,
    #         source_URL=config.source_URL,
    #         local_data_file=config.local_data_file,
    #         unzip_dir=config.unzip_dir
    #     )
    #     return data_ingestion 
    
    def get_data_validation_config(self) -> DataValidationConfig:
        create_directories([self.config.data_validation.root_dir])
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        data_validation = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema = schema
        )      
        return data_validation

In [15]:
class Data_Validation:
    def __init__(self,config: DataValidationConfig):
        self.config = config
        
    def validate_data(self):
        create_directories([self.config.root_dir])
        data = pd.read_csv(self.config.unzip_data_dir)
        all_cols = list(data.columns)
        
        self.validation_status = None
        
        for col in self.config.all_schema.keys():
            if col in all_cols:
                validation_status = True
            else:
                validation_status = False
                return validation_status
            
        return validation_status
    
    def save_status_file(self):
        validation_status = self.validate_data()
        
        with open(self.config.STATUS_FILE,"w") as file:
            file.write(f"validation_status : {validation_status}")

In [16]:
try: 
    config_manager = Configuration_Manager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = Data_Validation(data_validation_config)
    data_validation.save_status_file()
except Exception as e:
    raise e

[2024-10-29 08:20:49,051: INFO: common: yaml file at config/config.yaml loaded successfully]
[2024-10-29 08:20:49,053: INFO: common: yaml file at params.yaml loaded successfully]
[2024-10-29 08:20:49,055: INFO: common: yaml file at schema.yaml loaded successfully]
[2024-10-29 08:20:49,056: INFO: common: Directory created at artifacts]
[2024-10-29 08:20:49,057: INFO: common: Directory created at artifacts/data_validation]
[2024-10-29 08:20:49,057: INFO: common: Directory created at artifacts/data_validation]
